In [1]:
TARGET = "/Users/damoncrockett/Desktop/TVCG/"

In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv("./all.csv")

In [4]:
df = df[['local_path','IG_city_name','Mean Saturation','Mean Brightness','Exposure Balance','Uniqueness']]

In [5]:
df = df[df.Uniqueness.notnull()]

In [6]:
df.reset_index(drop=True,inplace=True)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99999 entries, 0 to 99998
Data columns (total 6 columns):
local_path          99999 non-null object
IG_city_name        99999 non-null object
Mean Saturation     99999 non-null float64
Mean Brightness     99999 non-null float64
Exposure Balance    99999 non-null float64
Uniqueness          99999 non-null float64
dtypes: float64(4), object(2)
memory usage: 5.3+ MB


In [8]:
cities = ['bangkok', 'berlin', 'moscow', 'sao', 'tokyo']

In [21]:
features = list(df.columns)[2:]

In [22]:
from shapely.geometry import Point
from PIL import Image

In [33]:
thumb_side = 100

In [34]:
sample_size = 1000

In [35]:
plot_wide = 180
plot_tall = 36

In [36]:
px_w = plot_wide * thumb_side
px_h = plot_tall * thumb_side

In [37]:
exemplar_h = int(round(plot_tall/2))
moscow = int(round(plot_wide/2))
berlin = moscow - plot_tall
bangkok = berlin - plot_tall
sao = moscow + plot_tall
tokyo = sao + plot_tall
        
d = {
'moscow':int(round(plot_wide/2)),
'berlin':moscow - plot_tall,
'bangkok':berlin - plot_tall,
'sao':moscow + plot_tall,
'tokyo':sao + plot_tall
}

In [38]:
for feature in features:
    x,y = range(plot_wide) * plot_tall, np.repeat(range(plot_tall),plot_wide)
    grid_list = pd.DataFrame(x,columns=['x'])
    grid_list['y'] = y
        
    point = []
    m = len(grid_list.index)
    for i in range(m):
        point.append(Point(grid_list.x.loc[i],grid_list.y.loc[i]))
    grid_list['point'] = point
    open_grid = list(grid_list.point)
        
    canvas = Image.new('RGB',(px_w, px_h),(50,50,50))
    
    for city in cities:
        tmp = df[df.IG_city_name==city]
        tmp = tmp.sample(n=sample_size)
        tmp.sort(feature,inplace=True)
        tmp.reset_index(drop=True,inplace=True)
        
        exemplar = Point(d[city],exemplar_h)
        im = Image.open(tmp.local_path.loc[0])
        im.thumbnail((thumb_side,thumb_side),Image.ANTIALIAS)
        x = int(exemplar.x) * thumb_side
        y = int(exemplar.y) * thumb_side
        canvas.paste(im,(x,y))
        open_grid.remove(exemplar)
            
        for i in range(1,len(tmp)):
            im = Image.open(tmp.local_path.loc[i])
            im.thumbnail((thumb_side,thumb_side),Image.ANTIALIAS)
            closest_open = min(open_grid,key=lambda x: exemplar.distance(x))
            x = int(closest_open.x) * thumb_side
            y = int(closest_open.y) * thumb_side
            canvas.paste(im,(x,y))
            open_grid.remove(closest_open)
        
        print city
    
    canvas.save(TARGET+feature+".png")

bangkok
berlin
moscow
sao
tokyo
bangkok
berlin
moscow
sao
tokyo
bangkok
berlin
moscow
sao
tokyo
bangkok
berlin
moscow
sao
tokyo
